In [3]:
import psycopg2
import json
import pandas as pd
from pandas import DataFrame
import os
import glob
import datetime
from pandas import ExcelWriter
from Bucketsize import nodule_size,connect_to_db,liver_size

os.chdir('/Users/kinnaripatel/Desktop/k')
path = "/Users/kinnaripatel/Desktop/k"

for filename in glob.glob(os.path.join(path, '*.json')):     
    with open(filename, mode='r') as f:
        data = json.load(f)
        connection = connect_to_db(data['site_info']['db_name'])
        print (data['site_info']['has_facilities'])
        cursor = connection.cursor()
        if data['site_info']['has_facilities'] == True:
            for facility in data['facility_info']:
                postgreSQL_select_Query_with_facility = """select messages."messageId","{}","dateOfBirth","patientVisitFacility","{}","{}","{}","{}","report","has_measure","max_measure","laterality","max_measure_location","max_nodule_shape","max_nodule_calcification","max_nodule_density","max_nodule_margin","max_nodule_lobe","single_multiple_abnormality" 
from messages inner join liver as ipn on messages."messageId" = ipn."messageId" 
where report is not null and "esIndex" ilike '{}' and  "{}" >=  CURRENT_DATE - INTERVAL '12 months'""".format(facility['mrn_column_id'],facility['procedure_desc_column_id'],facility['procedure_code_column_id'],facility['exam_date_column_id'],facility['accession_column_id'],facility['facility_id'], facility['exam_date_column_id'])

                
                cursor.execute(postgreSQL_select_Query_with_facility)
                print("Selecting rows from exams table")
                exams = cursor.fetchall()
                df=pd.DataFrame(list(exams))
                file_name = facility['facility_id']+'.xlsx'
                df.columns =["ID","MRN","DateOfBirth","Facility","Procedure Description","Procedure Code","Exam Date","Accession Number","report","has_measure","max_measure","laterality","max_measure_location","max_nodule_shape","max_nodule_calcification","max_nodule_density","max_nodule_margin","max_nodule_lobe","single_multiple_abnormality"]
                #df.to_csv('sample'+facility['facility_id']+'.csv')
                #df['calc']= df['max_measure'].apply[nodule_size]
                df['size_category'] = df["max_measure"].apply(liver_size)
                print(df['size_category'])
                df1=df.groupby('size_category')['size_category'].count().reset_index(name="count")
                print(df1)
                #df1.to_csv('sample1'+facility['facility_id']+'.csv')
                patient_df=df.sort_values('Exam Date').drop_duplicates(subset=['MRN'], keep='last')
               # patient_df.to_csv('patientsample'+facility['facility_id']+'.csv')
                writer = pd.ExcelWriter(file_name, engine='xlsxwriter')
                df.to_excel(writer,sheet_name = 'exams', index=False)
                df1.to_excel(writer,sheet_name = 'sizebucket',index=False)
                patient_df.to_excel(writer,sheet_name = 'patients', index=False)
                writer.save() 
        else:
            postgreSQL_select_Query = """select messages."messageId","{}","dateOfBirth","patientVisitFacility","{}","{}","{}","{}","report","has_measure","max_measure","laterality","max_measure_location","max_nodule_shape","max_nodule_calcification","max_nodule_density","max_nodule_margin","max_nodule_lobe","single_multiple_abnormality" 
from messages inner join liver as ipn on messages."messageId" = ipn."messageId" 
where report is not null and  "{}" >=  '2021-05-01' and "{}" < '2021-06-01' """.format(data['site_info']['mrn_column_id'],data['site_info']['procedure_desc_column_id'],data['site_info']['procedure_code_column_id'],data['site_info']['exam_date_column_id'],data['site_info']['accession_column_id'],data['site_info']['exam_date_column_id'],data['site_info']['exam_date_column_id'])

            cursor.execute(postgreSQL_select_Query)
            print("Selecting rows from exams table")
            exams1 = cursor.fetchall()
            df2=pd.DataFrame(list(exams1))
            file_name = data['site_info']['site_id']+'.xlsx'
            df2.columns =["ID","MRN","DateOfBirth","Facility","Procedure Description","Procedure Code","Exam Date","Accession Number","report","has_measure","max_measure","laterality","max_measure_location","max_nodule_shape","max_nodule_calcification","max_nodule_density","max_nodule_margin","max_nodule_lobe","single_multiple_abnormality"]
            #df2.to_csv('sample'+filename+'.csv')
            df2['size_category'] = df2["max_measure"].apply(liver_size)
            #print(df['size_category'])
            df3=df2.groupby('size_category')['size_category'].count().reset_index(name="count")
            print(df3)
            #df3.to_csv('sample1'+filename+'.csv')
            patient_df1=df2.sort_values('Exam Date').drop_duplicates(subset=['MRN'], keep='last')
            #patient_df1.to_csv('patientsample'+filename+'.csv')
            writer = pd.ExcelWriter(file_name, engine='xlsxwriter')
            df2.to_excel(writer,sheet_name = 'exams', index=False)
            df3.to_excel(writer,sheet_name = 'sizebucket', index=False)
            patient_df1.to_excel(writer,sheet_name = 'patients', index=False)
            writer.save() 

False
Selecting rows from exams table
  size_category  count
0       1.0-1.5     27
1          <1cm     39
2        >1.5cm     68
3      negative   1725


/var/folders/95/66y8yr891nx_wrg5vxt9ppy00000gp/T/ipykernel_14785/1526797899.py:70: FutureWarning: save is not part of the public API, usage can give in unexpected results and will be removed in a future version
  writer.save()


True
Selecting rows from exams table
0       negative
1       negative
2       negative
3       negative
4       negative
          ...   
2721    negative
2722    negative
2723    negative
2724    negative
2725    negative
Name: size_category, Length: 2726, dtype: object
  size_category  count
0       1.0-1.5     35
1          <1cm     26
2        >1.5cm    113
3      negative   2552


/var/folders/95/66y8yr891nx_wrg5vxt9ppy00000gp/T/ipykernel_14785/1526797899.py:46: FutureWarning: save is not part of the public API, usage can give in unexpected results and will be removed in a future version
  writer.save()


Selecting rows from exams table
0        negative
1        negative
2        negative
3        negative
4        negative
           ...   
35718    negative
35719        <1cm
35720    negative
35721    negative
35722    negative
Name: size_category, Length: 35723, dtype: object
  size_category  count
0       1.0-1.5    120
1          <1cm     61
2        >1.5cm    308
3      negative  35234


/var/folders/95/66y8yr891nx_wrg5vxt9ppy00000gp/T/ipykernel_14785/1526797899.py:46: FutureWarning: save is not part of the public API, usage can give in unexpected results and will be removed in a future version
  writer.save()
